In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import Callback, EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
from keras import Model
from tensorflow.keras.optimizers import Adam
# System libraries
from pathlib import Path
import os.path
# Metrics
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
#check dataset
import os
import PIL
from PIL import Image
from pathlib import Path
from PIL import UnidentifiedImageError
dataset = "/home/bkcs/NIMA/dataset3/Low/"
path = Path(dataset).rglob("*.jpg")
for img_p in os.listdir(dataset):
    try:
        img = Image.open(os.path.join(dataset, img_p))
    except:
        os.remove(os.path.join(dataset, img_p))

In [ ]:
import numpy as np
import os
import glob

#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
'''
Checks all images from the AVA dataset if they have corrupted jpegs, and lists them for removal.

Removal must be done manually !
'''

base_images_path = '/home/bkcs/NIMA/dataset2/LowHigh/'
ava_dataset_path = '/home/bkcs/NIMA/AVA.txt'


#print(files)
train_image_paths = []
train_scores = []

print("Loading training set and val set")
with open(ava_dataset_path, mode='r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        token = line.split()
        id = int(token[1])

        values = np.array(token[2:12], dtype='float32')
        values /= values.sum()

        file_path = base_images_path + str(id) + '.jpg'
        if os.path.exists(file_path):
            train_image_paths.append(file_path)
            train_scores.append(values)

        count = 255000 // 20
        if i % count == 0 and i != 0:
            print('Loaded %0.2f of the dataset' % (i / 255000. * 100))


train_image_paths = np.array(train_image_paths)
train_scores = np.array(train_scores, dtype='float32')

# val_image_paths = train_image_paths[-50:]
# val_scores = train_scores[-50:]
# train_image_paths = train_image_paths[:-50]
# train_scores = train_scores[:-50]

def parse_data(filename):
    image = tf.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    return image

sess = tf.Session()
with sess.as_default():
    sess.run(tf.global_variables_initializer())

    count = 0
    fn = tf.placeholder(dtype=tf.string)
    img = parse_data(fn)

    for path in train_image_paths:
        try:
            sess.run(img, feed_dict={fn: path})
        except Exception as e:
            print(path, "failed to load !")
            count += 1
            os.remove(path)

    print(count, "images failed to load !")

print("All done !")

In [3]:
BATCH_SIZE = 32
TARGET_SIZE = (224, 224)
dataset = "/home/bkcs/NIMA/dataset2"
a = {"LowHigh": 0, "Medium":1}
def convert_path_to_df(dataset):
    labels = []
    image_dir = Path(dataset)

    # Get filepaths and labels
    filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.png')) + list(image_dir.glob(r'**/*.PNG'))

    labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
    # for i in labels1:
    #     labels.append(a[i])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    # Concatenate filepaths and labels
    image_df = pd.concat([filepaths, labels], axis=1)
    return image_df

image_df = convert_path_to_df(dataset)

In [4]:
image_df

,Filepath,Label
0,/home/bkcs/NIMA/dataset2/Medium/160409.jpg,Medium
1,/home/bkcs/NIMA/dataset2/Medium/446171.jpg,Medium
2,/home/bkcs/NIMA/dataset2/Medium/754457.jpg,Medium
3,/home/bkcs/NIMA/dataset2/Medium/44807.jpg,Medium
4,/home/bkcs/NIMA/dataset2/Medium/598245.jpg,Medium
...,...,...
249126,/home/bkcs/NIMA/dataset2/LowHigh/492636.jpg,LowHigh
249127,/home/bkcs/NIMA/dataset2/LowHigh/707126.jpg,LowHigh
249128,/home/bkcs/NIMA/dataset2/LowHigh/607641.jpg,LowHigh
249129,/home/bkcs/NIMA/dataset2/LowHigh/766089.jpg,LowHigh


In [ ]:
from sklearn.utils import resample
def apply_downsampling(image_df):
    # Separate majority and minority classes
    
    majority_df = image_df[image_df['Label'] == 'Medium']
    minority_df_Low = image_df[image_df['Label'] == "Low"]
    minority_df_High = image_df[image_df['Label'] == "High"]

    # Oversample the minority class to have the same number of instances as the majority class
    downsampled_majority_df_1 = resample(majority_df ,n_samples=len(minority_df_Low), replace=True, random_state=42)
    #oversampled_minority_df_2 = resample(minority_df_High, n_samples=len(majority_df), replace=True, random_state=42)
    # Combine the oversampled minority class and the majority class
    oversampled_df = pd.concat([minority_df_Low,minority_df_High, downsampled_majority_df_1, ])

    return oversampled_df
#image_df = apply_downsampling(image_df)

In [ ]:
image_df

In [4]:
from sklearn.utils import resample
def hybrid_sampling(image_df):
    # Separate majority and minority classes
    
    majority_df = image_df[image_df['Label'] == 'Medium']
    minority_df_LowHigh = image_df[image_df['Label'] == "LowHigh"]
   
    n_samples = (len(majority_df)+len(minority_df_LowHigh))//2
    # Oversample the minority class to have the same number of instances as the majority class
    downsampled_majority_df = resample( majority_df ,n_samples=n_samples, replace=True, random_state=42)
    oversampled_minority_df_1 = resample(minority_df_LowHigh, n_samples=n_samples, replace=True, random_state=42)
    # Combine the oversampled minority class and the majority class
    oversampled_df = pd.concat( [oversampled_minority_df_1, downsampled_majority_df ])

    return oversampled_df

In [5]:
# Separate in train and test data
train_df, test_df = train_test_split(image_df, test_size=0.2, shuffle=True, random_state=42)

train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2
)
val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    subset='validation'
)
#train_df = hybrid_sampling(train_df)

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)
# Split the data into three categories.
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    subset='training'
)



test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 39860 validated image filenames belonging to 2 classes.
Found 159444 validated image filenames belonging to 2 classes.
Found 49827 validated image filenames belonging to 2 classes.


In [6]:
from tensorflow import keras
import numpy as np
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
lowhigh = image_df['Label'].value_counts()['LowHigh']
print(lowhigh)
medium = image_df['Label'].value_counts()['Medium']
print(medium)
weight_for_medium = (1 / medium) * (len(image_df) / 2.0) 
print(weight_for_medium)
weight_for_lowhigh = (1 / lowhigh) * (len(image_df) / 2.0)
print(weight_for_lowhigh)
class_weight = {0: weight_for_lowhigh, 1: weight_for_medium}

2023-07-11 19:36:17.763407: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


11000
238131
0.5230965309010587
11.324136363636363


2023-07-11 19:36:17.802812: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-11 19:36:17.803003: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-11 19:36:17.803942: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-11 19:36:17.804596: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [7]:
# Load the pretained model
pretrained_model = tf.keras.applications.efficientnet.EfficientNetB4(
    input_shape=(224, 224, 3),
    weights = 'imagenet',
    include_top=False,
    pooling='max'
)

pretrained_model.trainable = True
# Create checkpoint callback
checkpoint_path = "/home/bkcs/NIMA/weights/EfficientNetB4_classification_weights_cls_weight_2cls_1(official2).h5"
checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      save_weights_only=True,
                                      monitor="val_prc",
                                      save_best_only=True)

# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = EarlyStopping(monitor = "val_prc", # watch the val loss metric
                               patience = 15,
                               restore_best_weights = True) 
# if val loss decreases for 3 epochs in a row, stop training

reduce_lr = ReduceLROnPlateau(monitor='val_prc', factor=0.2, patience=3, min_lr=1e-6)

In [8]:
#augment
augment = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Resizing(224,224),
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.1)
])
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [9]:
inputs = pretrained_model.input
x = augment(inputs)
x = Dense(128, activation='relu')(pretrained_model.output)
x = BatchNormalization()(x)
x = Dropout(0.45)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.45)(x)


outputs = Dense(2, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)
model.load_weights('/home/bkcs/NIMA/weights/EfficientNetB4_classification_weights_cls_weight_2cls_1(official).h5')

model.compile(
    optimizer=Adam(0.0001),
    loss='categorical_crossentropy',
    metrics=METRICS
)
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=100,
    callbacks=[
        early_stopping,
        checkpoint_callback,
        reduce_lr
    ],
    class_weight=class_weight
)

Epoch 1/100


2023-07-11 19:36:47.292663: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2023-07-11 19:36:48.237167: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.79GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-07-11 19:36:48.373160: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.60GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-07-11 19:36:48.380037: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.63GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains

  33/4983 [..............................] - ETA: 34:58 - loss: 0.1530 - tp: 974.0000 - fp: 82.0000 - tn: 974.0000 - fn: 82.0000 - accuracy: 0.9223 - precision: 0.9223 - recall: 0.9223 - auc: 0.9736 - prc: 0.9712

KeyboardInterrupt: 

Classify 2 classes: LowHigh and Medium (VGG19)

In [25]:
! pip install tensorflow==2.10


/bin/bash: /home/bkcs/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [1]:
import tensorflow as tf
print(tf.__version__)


2023-07-11 19:35:46.403321: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-11 19:35:46.552398: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-11 19:35:47.216169: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/bkcs/anaconda3/lib/:/home/bkcs/anaconda3/lib/python3.10/site-packages/nvidia/cudnn/lib:/home/bkcs/anaconda3/lib/:/home/bkcs/anaconda3/lib/python3.10/site-packages/nvidia/cudnn/lib:/home/bkcs/anaconda3/lib/

2.10.0


In [13]:
! pip install keras==2.10

/bin/bash: /home/bkcs/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.8.0 requires keras<2.9,>=2.8.0rc0, but you have keras 2.10.0 which is incompatible.
tensorflow-gpu 2.8.0 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.1 which is incompatible.


In [ ]:
# from keras.models import Model
# pretrained_model = tf.keras.applications.vgg19.VGG19(
#     input_shape=(224, 224, 3),
#     # weights = 'imagenet',
#     include_top=False,
#     pooling='max'
# )
# inputs = pretrained_model.input
# x = Dense(128, activation='relu')(pretrained_model.output)
# x = BatchNormalization()(x)
# x = Dropout(0.45)(x)
# x = Dense(256, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.45)(x)
# outputs = Dense(3, activation='softmax')(x)

# model = Model(inputs=inputs, outputs=outputs)

# model.load_weights('/home/bkcs/NIMA/weights/VGG19_classification_weights_hybrid_2cls(official).h5')
# model.compile(
#     optimizer=Adam(0.00001),
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )
results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [ ]:
import numpy as np
# Predict the label of the test_images
pred = model.predict(test_images)
pred = np.argmax(pred,axis=1)

# Map the label
labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]


In [ ]:
train_images.class_indices

In [ ]:
y_test = list(test_df.Label)
print(classification_report(y_test, pred))

In [ ]:
confusion_matrix(y_test, pred, labels=list(labels.values()))

Classify 2 classes: Low and High (VGG19)

In [ ]:
results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [ ]:
import numpy as np
# Predict the label of the test_images
pred = model.predict(test_images)
pred = np.argmax(pred,axis=1)

# Map the label
labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

In [ ]:
y_test = list(test_df.Label)
print(classification_report(y_test, pred))

In [ ]:
confusion_matrix(y_test, pred, labels=list(labels.values()))

Classify 2 classes: Low and High (INceptionResnet)

In [ ]:
results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [ ]:
import numpy as np
# Predict the label of the test_images
pred = model.predict(test_images)
pred = np.argmax(pred,axis=1)

# Map the label
labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

In [ ]:
y_test = list(test_df.Label)
print(classification_report(y_test, pred))

In [ ]:
confusion_matrix(y_test, pred, labels=list(labels.values()))

Classify 2 classes: LowHigh and Medium (EfficientnetB4)

In [12]:
from keras.models import Model
pretrained_model = tf.keras.applications.efficientnet.EfficientNetB4(
    input_shape=(224, 224, 3),
    weights = 'imagenet',
    include_top=False,
    pooling='max'
)
inputs = pretrained_model.input
x = augment(inputs)
x = Dense(128, activation='relu')(pretrained_model.output)
x = BatchNormalization()(x)
x = Dropout(0.45)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.45)(x)
outputs = Dense(2, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)
model.load_weights('/home/bkcs/NIMA/weights/EfficientNetB4_classification_weights_cls_weight_2cls_1(official2).h5')
\
model.compile(
    optimizer=Adam(0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))
print(results)

    Test Loss: 0.37953
Test Accuracy: 88.75%
[0.3795323669910431, 0.8874906897544861]


2023-06-01 05:53:56.747976: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.61GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-06-01 05:53:56.766949: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.62GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


In [13]:
import numpy as np
# Predict the label of the test_images
pred = model.predict(test_images)
pred = np.argmax(pred,axis=1)

# Map the label
labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

In [14]:
y_test = list(test_df.Label)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

     LowHigh       0.16      0.36      0.22      2194
      Medium       0.97      0.91      0.94     47633

    accuracy                           0.89     49827
   macro avg       0.56      0.64      0.58     49827
weighted avg       0.93      0.89      0.91     49827



In [15]:
confusion_matrix(y_test, pred, labels=list(labels.values()))

array([[  792,  1402],
       [ 4204, 43429]])

Classify 2 classes: Low and High (EfficientnetB4)

In [ ]:
results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [ ]:
import numpy as np
# Predict the label of the test_images
pred = model.predict(test_images)
pred = np.argmax(pred,axis=1)

# Map the label
labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

In [ ]:
y_test = list(test_df.Label)
print(classification_report(y_test, pred))

In [ ]:
confusion_matrix(y_test, pred, labels=list(labels.values()))